In [3]:
!pip install --disable-pip-version-check -q awswrangler==2.14.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.0.11 requires pandas>=2, but you have pandas 1.3.5 which is incompatible.
dask-expr 1.0.11 requires pyarrow>=7.0.0, but you have pyarrow 6.0.1 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [4]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

In [5]:
import boto3
import sagemaker
import pandas as pd
from pyathena import connect

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

s3 = boto3.resource('s3')
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [6]:
s3_private_path_csv = "s3://{}/FinalProjectDataset/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-652903355321/FinalProjectDataset/csv


In [7]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
print(s3_staging_dir)

s3://sagemaker-us-east-1-652903355321/athena/staging


In [14]:
s3.Object(bucket, 'FinalProjectDataset/csv/olist_customers_dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_customers_dataset.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_geolocation_dataset1.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_geolocation_dataset1.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_geolocation_dataset2.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_geolocation_dataset2.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_order_items_dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_order_items_dataset.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_order_payments_dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_order_payments_dataset.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_order_reviews_dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_order_reviews_dataset.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_orders_dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_orders_dataset.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_products_dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_products_dataset.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/olist_sellers_dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/olist_sellers_dataset.csv', 'rb'))
s3.Object(bucket, 'FinalProjectDataset/csv/product_category_name_translation.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/ProjectDataset/product_category_name_translation.csv', 'rb'))

{'ResponseMetadata': {'RequestId': '467T7VJRECBK366D',
  'HostId': 'OxG7KSe+mYPBbr2V4mv/AbAqZDFtBgA4J+mOLOE+z1TqIxAK2rsdMGMc/FEbtkAZvVWRYiPBxc4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'OxG7KSe+mYPBbr2V4mv/AbAqZDFtBgA4J+mOLOE+z1TqIxAK2rsdMGMc/FEbtkAZvVWRYiPBxc4=',
   'x-amz-request-id': '467T7VJRECBK366D',
   'date': 'Sun, 12 May 2024 13:30:54 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4196d142e8f2b9697521fc50c97f626b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"4196d142e8f2b9697521fc50c97f626b"',
 'ServerSideEncryption': 'AES256'}

In [15]:
!aws s3 ls $s3_private_path_csv/

2024-05-12 13:30:51    9033957 olist_customers_dataset.csv
2024-05-12 13:30:52   23400724 olist_geolocation_dataset1.csv
2024-05-12 13:30:52   23582346 olist_geolocation_dataset2.csv
2024-05-12 13:30:53   15438671 olist_order_items_dataset.csv
2024-05-12 13:30:53    5777138 olist_order_payments_dataset.csv
2024-05-12 13:30:53   14451670 olist_order_reviews_dataset.csv
2024-05-12 13:30:53   17654914 olist_orders_dataset.csv
2024-05-12 13:30:54    2379446 olist_products_dataset.csv
2024-05-12 13:30:54     174703 olist_sellers_dataset.csv
2024-05-12 13:30:54       2613 product_category_name_translation.csv


In [16]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>